In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi
from pyspark.sql import SparkSession
from datetime import date

#create API connection and authenticate
api = KaggleApi()
api.authenticate()

#download dataset 
api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Daily_Country_Wise_Confirmed_Cases.csv')
api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Monkey_Pox_Cases_Worldwide.csv')

#create spark session
spark = SparkSession.builder.getOrCreate()

#create spark df
daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

def date_check():
    today = date.today()
    return str(today)

def column_grab():
    try:
        daily_df = daily_df.select(daily_df['Country'],daily_df['2022-07-01'])
        return daily_df
    except:
        pass

(daily_df[-1]).show(1)


TypeError: 'Column' object is not callable

In [ ]:
from airflow import DAG
from airflow.decorators import dag, task

#create our tasks
@task
def api_pull():
    """
    pulls the updated file from Kaggle
    """
    api = KaggleApi()
    api.authenticate()
    #download dataset 
    api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Daily_Country_Wise_Confirmed_Cases.csv')
    #create spark session
    spark = SparkSession.builder.getOrCreate()



@task
def main():
    """
    main.py, update table with new data, do transformations
    """

@task
def load():
    """
    load to bigqquery
    """

#set dag attributes
@dag(
    schedule_interval='@daily',
    start_date=datetime.utcnow(),
    catchup=False,
)
#function that calls all of our tasks to create dag
def monkeypox_dag():


![image](DAG.png)

In [46]:
# daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)
#pivot


def column_grab():
    daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)
    daily_df = daily_df.select(daily_df['Country'],daily_df['2022-07-01'])
    return daily_df


daily_df = column_grab()

daily_df.show(10)

dataset_df = spark.read.csv('Monkey_Pox_Cases_Worldwide.csv', header=True)

dataset_df = dataset_df.join(daily_df,['Country'],how='left')

dataset_df.withColumn('Confirmed_Cases', dataset_df['Confirmed_Cases'] + dataset_df['2022-07-01']).drop(dataset_df['2022-07-01']).show(10)

+-------------+----------+
|      Country|2022-07-01|
+-------------+----------+
|      England|         0|
|     Portugal|         0|
|        Spain|         0|
|United States|        64|
|      Germany|        85|
|      Belgium|         0|
|       Sweden|         0|
|        Italy|        33|
|       Canada|         0|
|       France|         0|
+-------------+----------+
only showing top 10 rows

+-------------+---------------+---------------+------------+------------------+-----------------+
|      Country|Confirmed_Cases|Suspected_Cases|Hospitalized|Travel_History_Yes|Travel_History_No|
+-------------+---------------+---------------+------------+------------------+-----------------+
|      England|         1185.0|            0.0|         5.0|               2.0|              7.0|
|     Portugal|          402.0|            0.0|         0.0|               0.0|             34.0|
|        Spain|         1196.0|            0.0|        11.0|               2.0|              0.0|
|United 

In [ ]:
import quinn

daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

def dashes_to_underscores(s):
    return s.replace("-", "_")


dataset_df = daily_df.transform(quinn.with_columns_renamed(dashes_to_underscores))

pandas_DF = dataset_df.toPandas()

pandas_DF.set_index('Country', inplace=True)
transpose_df = pandas_DF.transpose()

transpose_df.reset_index(inplace=True)

spark_df=spark.createDataFrame(transpose_df)
spark_df = spark_df.withColumnRenamed('index', 'Date')

spark_df.show()




In [107]:
daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

def date_check():
    today = date.today()
    return str(today)

def column_grab():
    try:
        daily_df = daily_df.select(daily_df['Country'],daily_df['2022-07-01'])
        return daily_df
    except:
        pass

test_df = daily_df.select('Country', daily_df['2022-06-24'])

daily_df = daily_df.select('Country',daily_df[-1])


pandas_df = daily_df.toPandas()

pandas_df.set_index('Country', inplace=True)
transpose_df = pandas_df.transpose()


transpose_df.reset_index(inplace=True)

spark_df=spark.createDataFrame(transpose_df)
spark_df = spark_df.withColumnRenamed('index', 'Date')

print(len(spark_df.columns))
print(len(test_df.columns))

def column_grab():
    len(current)

57
2


In [159]:
from datetime import datetime
from pyspark.sql.functions import lit

df = spark.createDataFrame([
    ('7-1', 1, 2, 0),
    ('7-2',2, 4, 3),
    ('7-3',3, 6, 0)
], schema='Date string, USA string, Canada string, Mexico string')

day_df = spark.createDataFrame([
    ('7-4',6, 6, 0, 1)
], schema='Date string, USA string, Canada string, Mexico string, Peru string')

def add_col(df1, df2):
    if len(df1.columns) < len(df2.columns):
        for column in [column for column in df2.columns if column not in df1.columns]:
            df1 = df1.withColumn(column, lit(0))
            return df1

df_shape = df.count(), len(df.columns)

df_shape
# for column in [column for column in day_df.columns
#                if column not in df.columns]:
#     df = df.withColumn(column, lit(0))
# df = add_col(df, day_df)
# df.union(day_df).show()

(3, 4)

Script for daily pull

In [ ]:
def add_row(daily_df, rows):
    """
    Add rows to dataset but no new columns
    """
    #grab number of new rows
    daily_df = daily_df.select('Country',daily_df[rows])
    #create pandas df and set index
    pandas_df = daily_df.toPandas()
    pandas_df.set_index('Country', inplace=True)
    #transpose and reset index
    transpose_df = pandas_df.transpose()
    transpose_df.reset_index(inplace=True)
    #create spark df 
    spark_df=spark.createDataFrame(transpose_df)
    spark_df = spark_df.withColumnRenamed('index', 'Date')
    #write to csv? or update? 

def add_column(daily_df, rows):
    """
    Add rows and columns to dataset
    """
    #grab number of new rows
    daily_df = daily_df.select('Country',daily_df[rows])
    #create pandas df and set index
    pandas_df = daily_df.toPandas()
    pandas_df.set_index('Country', inplace=True)
    #transpose and reset index
    transpose_df = pandas_df.transpose()
    transpose_df.reset_index(inplace=True)
    #create spark df 
    spark_df=spark.createDataFrame(transpose_df)
    spark_df = spark_df.withColumnRenamed('index', 'Date')
    #pull dataset down from GCS??
    df = df
    for column in [column for column in spark_df.columns if column not in df.columns]:
        df = df.withColumn(column, lit(0))
    df = add_col(df, spark_df)
    df.union(spark_df)
    #write to csv? overwrite table?

In [ ]:
def update(shape_xcom):
    """
    Main function to create df of new data
    """
    daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)
    daily_shape = (len(daily_df.columns), daily_df.count())
    if daily_shape == shape_xcom:
        pass
    elif daily_shape[0] == shape_xcom[1]:
        rows = daily_df.count() - shape_xcom[0]
        add_row(daily_df, rows)
    else:
        rows = daily_df.count() - shape_xcom[0]
        add_column(daily_df, rows)


